<a href="https://colab.research.google.com/github/LM1997610/Data-Mining/blob/main/HW_2_DMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# DMT 2023 - Homework 2

## Group composition:

<br>

------------YOUR TEXT STARTS HERE------------

<br>

- Mazzucco, Luca, 1997610 \
- Zilviano, Paolo, 1916518

<div style="page-break-after: always; visibility: hidden"> 
\pagebreak
</div>

## Homework 2
The homework consists of two parts:

  1. PageRank
  
  2. Recommendation System

> Ensure that the notebook can be faithfully reproduced by anyone (hint: pseudo random number generation).

> If you need to set a random seed, set it to `24`.

# Part 1
In this part of the homework, you have to deal with the PageRank algorithm.

Import **ALL** the Python packages that you need for Part 1.

In [ ]:
#REMOVE_OUTPUT#
%%capture
!pip install --upgrade --no-cache-dir gdown
!pip install scikit-network

#YOUR CODE STARTS HERE#

import numpy as np; import pandas as pd
from bs4 import BeautifulSoup
from sknetwork.utils import get_neighbors
from sknetwork.ranking import PageRank
from tqdm import tqdm; from tabulate import tabulate

#YOUR CODE ENDS HERE#
#THIS IS LINE 15#

## Part 1.1
The data you need to process comes from the book *Le Morte D'Arthur* by Thomas Malory.
The dataset you need to build should be an unweighted and undirected graph, where nodes represent characters from the book and an edge connects two characters in the graph if their names appeared at least one time in the same chapter.

Using this dataset, you must then run various PageRank algorithms.

### 1.1.1
Download the data from the Drive link

In [ ]:
#REMOVE_OUTPUT#
!gdown 1zHgvidy9FvhZvE68S0mXWkoF-hHMpiUL
!gdown 1VjpTkFcbfaLIi4TXVafokW9e_bvGnfut

Downloading...
From: https://drive.google.com/uc?id=1zHgvidy9FvhZvE68S0mXWkoF-hHMpiUL
To: /content/The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory.html
100% 964k/964k [00:00<00:00, 537MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VjpTkFcbfaLIi4TXVafokW9e_bvGnfut
To: /content/The Project Gutenberg eBook of Le Morte D’Arthur, Volume II (of II), by Thomas Malory.html
100% 1.10M/1.10M [00:00<00:00, 530MB/s]


### 1.1.2
Parse the HTML. **Part** of code already provided: follow the comments to complete the code.


In [ ]:
with open('The Project Gutenberg eBook of Le Morte D’Arthur, Volume I (of II), by Thomas Malory.html') as fp:
    vol1 = BeautifulSoup(fp, 'html.parser')
with open('The Project Gutenberg eBook of Le Morte D’Arthur, Volume II (of II), by Thomas Malory.html') as fp:
    vol2 = BeautifulSoup(fp, 'html.parser')

def clean_text(txt):
    words_to_put_space_before = [".",",",";",":","’","'"]
    words_to_lowercase = ["First","How","Some","Yet","Of","A","The","What","Fifth"]
    
    app = txt.replace("\n"," ")
    for word in words_to_put_space_before:
        app = app.replace(word," "+word)
    for word in words_to_lowercase:
        app = app.replace(word+" ",word.lower()+" ")
    return app.strip()

def parse_html(soup):
    titles = []
    texts = []
    for chapter in soup.find_all("h3"):
        chapter_title = chapter.text
        if "CHAPTER" in chapter_title:
            chapter_title = clean_text("".join(chapter_title.split(".")[1:]))
            titles.append(chapter_title)
            
            chapter_text = [p.text for p in chapter.findNextSiblings("p")]
            chapter_text = clean_text(" ".join(chapter_text))
            texts.append(chapter_text)
    return titles, texts

In [ ]:
#YOUR CODE STARTS HERE#
#Extract all the chapters' titles and texts from the two volumes

titles_1, texts_1 = parse_html(vol1)
titles_2, texts_2 = parse_html(vol2)


num_doc = [str(i).zfill(1) for i in range(1,len(titles_1 + titles_2)+1)]



#Transform the list into a pandas DataFrame.

df = pd.DataFrame(zip(num_doc, titles_1+titles_2, texts_1+texts_2), 
                  columns=["docno", 'Titles', 'Texts'])

print("\ndf.shape:", df.shape)

#YOUR CODE ENDS HERE#
#THIS IS LINE 20#


df.shape: (503, 3)


Print the first 8 rows of the DataFrame.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#



a_table = df[["Titles", "Texts"]].apply(lambda x: x.str[:25]+"...").head(8)

print()
print(tabulate(a_table, headers='keys', tablefmt = "heavy_outline", numalign='center', stralign="center"))
print()



#YOUR CODE ENDS HERE#
#THIS IS LINE 10#


               Titles                        Texts
--  ----------------------------  ----------------------------
0   first , how Uther Pendrag...  It befell in the days of ...
1   how Uther Pendragon made ...  Then Ulfius was glad , an...
2   of the birth of King Arth...  Then Queen Igraine waxed ...
3   of the death of King Uthe...  Then within two years Kin...
4   how Arthur was chosen kin...  Then stood the realm in g...
5   how King Arthur pulled ou...  Now assay , said Sir Ecto...
6   how King Arthur was crown...  And at the feast of Pente...
7   how King Arthur held in W...  Then the king removed int...



<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.3
Extract character's names from the **titles** only. **Part** of code already provided: follow the comments to complete the code.

In [ ]:
all_characters = set()
def extract_character_names_from_string(string_to_parse):
    special_tokens = ["of","the","le","a","de"]

    remember = ""
    last_is_special_token = False

    tokens = string_to_parse.split(" ")
    characters_found = set()
    for i,word in enumerate(tokens):
        if word[0].isupper() or (remember!="" and word in special_tokens):
            #word = word.replace("'s","").replace("’s","")
            last_is_special_token = False
            if remember!="":
                if word in special_tokens:
                    last_is_special_token = True
                remember = remember+" "+word
            else: remember = word
        else:
            if remember!="":
                if last_is_special_token:
                    for tok in special_tokens:
                        remember = remember.replace(" "+tok,"")
                characters_found.add(remember)
            remember = ""
            last_is_special_token = False

    return characters_found

In [ ]:
#YOUR CODE STARTS HERE#
#Extract all characters' names

all_characters = df["Titles"].apply(extract_character_names_from_string)
#all_characters = set([x for x in all_characters if x[-2:]!="'s"])
all_characters = set.union(*map(set, all_characters)) 

print(" All_characters: ", all_characters, sep=" ")

#YOUR CODE ENDS HERE#
#THIS IS LINE 15#

 All_characters:  {'Lanceor', 'Bagdemagus', 'Romans', 'Anglides', 'Castle of Maidens', 'Sir Anguish', 'Sir Sagramore le Desirous', 'Alisander', 'Our Lord', 'Sir Galahad', 'Ireland', 'Sir Tristram', 'Sir Meliagaunce', 'King Anguish of Ireland', 'Archbishop of Canterbury', 'Saracens', 'Bragwaine', 'Sir Pedivere', 'King Arthur', 'Melias', 'Fair Maid of Astolat', 'King Pelleas', 'Uther Pendragon', 'Palamides', 'Dame Brisen', 'Almesbury', 'Elias', 'King Solomon', 'Sir Pelleas', 'Sir Bleoberis', 'Leodegrance', 'Ettard', 'Sir Galihodin', 'Sir Colgrevance', 'Sir Lamorak de Galis', 'Sir Lionel', 'Sir Nabon', 'Sir Gaheris', 'La Beale Isoud', 'Holy Sangreal', 'Sir Urre', 'Castle Lonazep', 'Lucius', 'Sir Beaumains', 'King Pellam', 'Balan', 'Sir Turquine', 'Chapel Perilous', 'Guenever', 'Beale Pilgrim', 'Breuse Saunce Pité', 'Sir Galahalt', 'Sir Lancelot', 'Morgan', 'Sir Palomides', 'Constantine', 'Pentecost', 'Sir Persant of Inde', 'Elaine', 'Sir Launcelot', 'Sir Epinogris', 'France', 'Queen', 'Si

Print the names of all the kings (i.e. characters with `King` in their name).

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#

kings = { person for person in  all_characters  if person.lower().find("king") > -1  } 

print(" Kings:", kings, sep=" ")



#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

 Kings: {'King Leodegrance', 'King Mark', 'King Ban', 'King of the Land of Cameliard', 'King Pellinore', 'King Anguish of Ireland', 'King Bors', 'King', 'King Arthur', 'King Mordrains', 'Maimed King', 'King Pelleas', 'King Brandegore', 'King Solomon', 'King Pelles', 'King of England', 'King Mark of Cornwall', 'King Uriens', 'King Lot', 'King Bagdemagus', 'King Pellam', 'King Howel of Brittany', 'King Rience', 'King Evelake', 'King Lot of Orkney'}


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.4
Some names refer to the same characters (e.g. `'Arthur' = 'King Arthur'`). A function is provided to extract the disambiguation dictionary: each key represents a name and the value represents the true character name (e.g. `{'Arthur': 'King-Arthur', 'King': 'King-Arthur', 'Bedivere':'Sir Bedivere'}`). Disambiguation sets, i.e. a list with sets representing the multiple names of a single character, are also provided.

> There may be some mistakes, but it does not matter (e.g. `'Cornwall' = 'King of Cornwall'`)

In [ ]:
disambiguate_to = {}
for x in all_characters:
    for y in all_characters:
        if x in y and x!=y:
            if x in disambiguate_to:
                previous_y = disambiguate_to[x]
                if len(y)>len(previous_y): disambiguate_to[x] = y
            else:
                disambiguate_to[x] = y
disambiguate_to.update({"King": "King Arthur",
                        "King of England": "King Arthur",
                        "Queen": "Queen Guenever",
                        "Sir Lancelot": "Sir Launcelot"})

disambiguate_sets = []
for x,y in disambiguate_to.items():
    inserted = False
    for z in disambiguate_sets:
        if x in z or y in z:
            z.add(x); z.add(y)
            inserted = True
    if not inserted:
        disambiguate_sets.append(set([x,y]))

while True:
    to_remove,to_add = [],[]
    for i1,s1 in enumerate(disambiguate_sets[:-1]):
        for s2 in disambiguate_sets[i1+1:]:
            if len(s1.intersection(s2))>0:
                to_remove.append(s1)
                to_remove.append(s2)
                to_add.append(s1.union(s2))
    if len(to_add)>0:
        for rm in to_remove:
            disambiguate_sets.remove(rm)
        for ad in to_add:
            disambiguate_sets.append(ad)
    else: break

### 1.1.5
Prepare the dataset for the PageRank algorithm.

> It should be a Pandas DataFrame with two fields: `character_1`, `character_2`.

> Each row must contain two characters' names if they appear together in at least one chapter **text**.

> The relevant characters are only those extracted in Part 1.1.3.

> Keep in mind that some characters have alternative names, but they refer to the same character.

> The dataset must not contain repetitions.

In [ ]:
#YOUR CODE STARTS HERE#

lista_p = list(all_characters)

for i in range(len(lista_p)):
  for j in disambiguate_to.keys():
    if lista_p[i] == j:
      lista_p[i] = disambiguate_to[j]

all_characters = set(lista_p)
alll, c1 , c2 = [] , [] , [] 

for i in tqdm(all_characters):
  for j in all_characters:
    if i != j:
      ll = [i,j]
      ll.sort()
      for txt in texts_1 + texts_2:
        if i in txt and j in txt and ll not in alll:
          c1.append(i) , c2.append(j), alll.append(ll)

couples_together_df = pd.DataFrame({ 'character_1': c1, 'character_2': c2}).drop_duplicates() 

#YOUR CODE ENDS HERE#
#THIS IS LINE 30#

100%|██████████| 170/170 [01:05<00:00,  2.60it/s]


Print the rows of the dataset where `Sir Lamorak` appears.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#

cdf1 = couples_together_df.loc[couples_together_df['character_1'] == 'Sir Lamorak de Galis']
cdf2 = couples_together_df.loc[couples_together_df['character_2'] == 'Sir Lamorak de Galis']
print(); print("shape:", pd.concat([cdf1,cdf2]).shape); print()
print(tabulate(pd.concat([cdf1,cdf2]).head(), headers='keys', tablefmt = "heavy_outline", 
               numalign='center', stralign="center")); print()

#YOUR CODE ENDS HERE#
#THIS IS LINE 10#


shape: (76, 2)

         character_1        character_2
---  --------------------  --------------
913  Sir Lamorak de Galis    Sir Lionel
914  Sir Lamorak de Galis    Sir Nabon
915  Sir Lamorak de Galis   Sir Gaheris
916  Sir Lamorak de Galis  La Beale Isoud
917  Sir Lamorak de Galis  Castle Lonazep



<div style="page-break-after: always; visibility: hidden"> 
\pagebreak
</div>

### 1.1.6
Print the sorted list of all character names (without duplicates) in ascending alphabetical order.

Print also the length of this list.

In [ ]:
#YOUR CODE STARTS HERE#

print('Total number of characters:', len(all_characters), "\n")

all_characters = list(all_characters)

all_characters.sort()

print(" All_characters: ", all_characters, sep=" ")



#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Total number of characters: 170 

 All_characters:  ['Abbot', 'Alice', 'Alisander le Orphelin', 'Almaine', 'Almesbury', 'Andred', 'Anglides', 'Archbishop of Canterbury', 'Avoutres', 'Balan', 'Balin', 'Beale Pilgrim', 'Benwick', 'Boudwin', 'Bragwaine', 'Camelot', 'Carbonek', 'Carlion', 'Castle Lonazep', 'Castle of Maidens', 'Castle of Pendragon', 'Chapel Perilous', 'Christmas', 'Constantine', 'Corsabrin', 'Court', 'Dame Brisen', 'Dame Elaine', 'Damosel of the Lake', 'David', 'Dover', 'Excalibur', 'Fair Maid of Astolat', 'Feast of Pentecost', 'Forest Perilous', 'France', 'Garlon', 'God', 'Gouvernail', 'Great Royalty', 'Griflet', 'Helin le Blank', 'Holy Sangreal', 'Humber', 'Island', 'Joseph', 'Joyous Gard', 'Joyous Isle', 'Kehydius', 'King Anguish of Ireland', 'King Arthur', 'King Bagdemagus', 'King Ban', 'King Bors', 'King Brandegore', 'King Evelake', 'King Howel of Brittany', 'King Leodegrance', 'King Lot of Orkney', 'King Mark of Cornwall', 'King Mordrains', 'King Pellam', 'King Pelle

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.7
Create the adjacency matrix for the graph, assigning to each character a node identifier equal to the index that the character name has in ascending alphabetical order (remember that the first element of a list in Python has index 0).

In [ ]:
#YOUR CODE STARTS HERE#


nodes = {}


for i in range(len(all_characters)):

  nodes[i] = all_characters[i]

adj_matrix = np.zeros((len(all_characters), len(all_characters)))

for i in range(len(c1)):

    adj_matrix[all_characters.index(c1[i]), all_characters.index(c2[i])] += 1
    adj_matrix[all_characters.index(c2[i]), all_characters.index(c1[i])] += 1


#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

### 1.1.8
Compute the PageRank vector for the obtained graph using a damping factor of 0.85.

In [ ]:
#YOUR CODE STARTS HERE#


pagerank = PageRank(damping_factor=0.85, solver="piteration", n_iter=1000, tol=10**-6)
pagerank_vector = pagerank.fit_transform(adj_matrix)



#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Print the name and the PageRank score of the top-15 characters according to the PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#
top_15_char, pagerank_scores = [] , []
top_15 = sorted(enumerate(pagerank_vector), key=lambda x:x[1], reverse=True)[:15]
for i, rank in top_15:
  top_15_char.append(nodes[i])
  pagerank_scores.append(rank)
top_15_characters_df = pd.DataFrame({ 'character': top_15_char, 'PageRank score': pagerank_scores});print()
print(tabulate(top_15_characters_df, headers='keys', tablefmt = "heavy_outline", numalign='center', stralign="center")); print()
#YOUR CODE ENDS HERE#
#THIS IS LINE 10#


      character      PageRank score
--  --------------  ----------------
0    King Arthur       0.0208746
1        God           0.0204287
2   Sir Launcelot      0.0199087
3    Sir Gawaine       0.0157327
4   Queen Guenever     0.0143519
5     Sir Ector        0.0133868
6      Sir Bors        0.0131417
7      Sir Kay         0.0130416
8   Sir Palomides      0.0119588
9       Wales          0.0119072
10  La Beale Isoud      0.01182
11  Sir Percivale      0.0112442
12   Sir Gaheris       0.0107368
13     Camelot         0.0106813
14  Sir Bleoberis       0.010677



<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.9
Compute the Topic-specific PageRank vector for the obtained graph using a damping factor of 0.75, by considering as topic the *Queens*: a character belongs to the topic if its name starts with the string `Queen`.

In [ ]:
#YOUR CODE STARTS HERE#
top_related, queen_related = {} , set()
for i in range(len(adj_matrix)):
    top_related[nodes[i]] = set()
    for j in range(len(adj_matrix[i])):
        if adj_matrix[i][j] == 1:
            top_related[nodes[i]].add(j)    
for i in top_related:
  if 'Queen' in i:
    queen_related.update(top_related[i])
pagerank_2 = PageRank(damping_factor=0.75, solver="piteration", n_iter=1000, tol=10**-6)
topic_as_map__page_id__landing_probability = {}
set_of_page_ids_belonging_to_the_selected_topic = queen_related
n = len(set_of_page_ids_belonging_to_the_selected_topic)
for c_page_id_belonging_to_the_topic in set_of_page_ids_belonging_to_the_selected_topic:
    topic_as_map__page_id__landing_probability[c_page_id_belonging_to_the_topic] = 1. / n
pagerank_vector_2 = pagerank.fit_transform(adj_matrix, weights=topic_as_map__page_id__landing_probability)

#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Print the name and the PageRank score of the top-16 characters according to the Topic-specific PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#
top_16_char, pagerank_scores_2 = [] , []
top_16 = sorted(enumerate(pagerank_vector_2), key=lambda x:x[1], reverse=True)[:16]
for i, rank in top_16:
  top_16_char.append(nodes[i])
  pagerank_scores_2.append(rank)
top_16_characters_df = pd.DataFrame({ 'character': top_16_char, 'PageRank score': pagerank_scores_2});print()
print(tabulate(top_16_characters_df, headers='keys', tablefmt = "heavy_outline", numalign='center', stralign="center")); print()
#YOUR CODE ENDS HERE#
#THIS IS LINE 10#


      character      PageRank score
--  --------------  ----------------
0    King Arthur       0.0211749
1   Sir Launcelot      0.0210497
2        God           0.0206672
3   Queen Guenever     0.0168513
4    Sir Gawaine       0.0167249
5     Sir Ector        0.0149551
6      Sir Bors        0.0146892
7      Sir Kay         0.0143129
8   Sir Palomides      0.0134301
9   La Beale Isoud     0.0131346
10      Wales          0.0130464
11  Sir Percivale      0.0122893
12  Sir Bleoberis      0.0122103
13    Sir Lionel       0.0119693
14   Sir Dinadan       0.0119031
15   Sir Mordred        0.011883



<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.10
Compute the Personalized PageRank vector for the obtained graph using a damping factor of 0.2 for each of the *Knights*: a character belongs to the topic if its name starts with the string `Sir`.

In [ ]:
#YOUR CODE STARTS HERE#

def compute_personalized_pagerank_Knights(page_id):

  pagerank = PageRank(damping_factor=0.2, solver="piteration", n_iter=1000, tol=10 ** -6)
  map__page_id__landing_probability = {}
  map__page_id__landing_probability[page_id] = 1.
  pagerank_vector = pagerank.fit_transform(adj_matrix, weights=map__page_id__landing_probability)

  return pagerank_vector

id_dict = {}
for i in nodes.items():
    for j in range(len(i)):
        if j == 0:
            id_dict[i[1]] = i[0] 

map__page_id__landing_probability = {}    
sirs = {}
for i in id_dict.keys():

    if 'Sir' in i or 'Knight' in i:
        sirs[i] = id_dict[i]

knights_pprks = []
for page_id in sirs.values():
  knights_pprks.append(compute_personalized_pagerank_Knights(page_id))

#YOUR CODE ENDS HERE#
#THIS IS LINE 30#

For each of the *Knights*, print the name and the PageRank score of the top-2 characters according to the Personalized PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#

top_2_df = []
for j in knights_pprks:

  top_2_char, pagerank_scores_3 = [] , []
  top_2 = sorted(enumerate(j), key=lambda x:x[1], reverse=True)[:2]

  for i, rank in top_2:
    top_2_char.append(nodes[i])
    pagerank_scores_3.append(rank)

  top_2_characters_df = pd.DataFrame({ 'character': top_2_char, 'PageRank score': pagerank_scores_3})
  top_2_df.append(top_2_characters_df)

#for i in top_2_df:
#  display(i)

#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 1.1.11
Compute Topic-specific PageRank for the graph using a damping factor of 0.2. Imagine you are in an **online** context.

The Topic is *Knights* (list of characters defined in step 1.1.7)

In [ ]:
#YOUR CODE STARTS HERE#
list__map__page_id__PersonalizedPageRank_score = []
for j in knights_pprks:
  oc_dict = {}
  for i in range(len(j)):
    oc_dict[i] = j[i]
  list__map__page_id__PersonalizedPageRank_score.append(oc_dict)

aggregated_map__page_id__PersonalizedPageRank_score = {}
length = len(list__map__page_id__PersonalizedPageRank_score)
for map__page_id__PersonalizedPageRank_score in list__map__page_id__PersonalizedPageRank_score:
    for page_id, PersonalizedPageRank_score in map__page_id__PersonalizedPageRank_score.items():
        if page_id not in aggregated_map__page_id__PersonalizedPageRank_score:
            aggregated_map__page_id__PersonalizedPageRank_score[page_id] = 0.
        aggregated_map__page_id__PersonalizedPageRank_score[page_id] += PersonalizedPageRank_score / length

lll = list(aggregated_map__page_id__PersonalizedPageRank_score.items())

#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Print the name and the PageRank score of the top-8 characters according to the Topic-specific PageRank score of their associated nodes.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#

top_8_char, online_pagerank_scores = [nodes[lll[i][0]] for i in range(len(lll))] , [lll[j][1]for j in range(len(lll))]
opr_df = pd.DataFrame({ 'character': top_8_char, 'PageRank score': online_pagerank_scores})
top_8_characters_df = opr_df.sort_values('PageRank score', ascending = False).head(8)

print()
print(tabulate(top_8_characters_df, headers='keys', tablefmt = "heavy_outline", numalign='center', stralign="center"))
print()



#YOUR CODE ENDS HERE#
#THIS IS LINE 10#


             character            PageRank score
---  --------------------------  ----------------
136        Sir Launcelot            0.0185228
132         Sir Gawaine             0.0159042
134     Sir Lamorak de Galis        0.0154105
146        Sir Palomides            0.0154006
160          Sir Uriens             0.0151515
106         Sir Anguish             0.0151515
140         Sir Malgrin             0.0151515
69   Knight of the Black Launds     0.0151515



<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

## Part 1.2

### 1.2.1
Given a graph with $n$ nodes:
* Node $A$ is connected to all the other nodes.
* There are no other edges.

What will be the PageRank of node $A$?

Does the result depend on the damping factor or number of nodes $n$? If yes, please describe the value of PageRank as both vary.

**Use at most 3 sentences.**

----------YOUR TEXT STARTS HERE----------

In this case node A will have a PageRank of 1.
The PageRank value of node A does not depend on the damping factor or number of nodes n because it is connected to all other nodes in the graph.


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

# Part 2
In this part of the homework, you have to improve the performance of various recommendation-systems by using non-trivial algorithms and also by performing the tuning of the hyper-parameters.

Import here **ALL** the Python packages that you need for Part 2.

In [ ]:
#REMOVE_OUTPUT#
#YOUR CODE STARTS HERE#
%%capture
!pip install scikit-surprise

import multiprocessing

import pandas as pd
from tqdm import tqdm
from tabulate import tabulate

from surprise import *

#YOUR CODE ENDS HERE#
#THIS IS LINE 15#

## Part 2.1
Apply **all** algorithms for recommendation made available by “Surprise” libraries on the provided dataset:
* **with their default configuration**
* using **ALL** CPU-cores available on the remote machine by specifying the value in an **explicit** way with an integer number.

You also need to:
* use Alternating Least Squares as baselines estimation method
* use cosine similarity as similarity measure
* use item-item similarity
* if a number of iterations is to be set, it must be 25

> Not all options may be applicable to all algorithms

### 2.1.1
Prepare the dataset for the Recommendation algorithms.

> It should be a Pandas DataFrame with three fields: `Ruler`, `Knight`, `Rating`.

> Each row must contain two characters' names if they appear together in at least one chapter **text**.

> The relevant characters are only those extracted in Part 1.1.3.

> Keep in mind that some characters have alternative names, but they refer to the same character.

> The dataset must not contain repetitions.

Also:

> A `Ruler` is a character whose name starts with `King` or `Queen`.

> A `Knight` is a character whose ame starts with `Knight` or `Sir`.

> The `Rating` represents the number of chapters in which two characters appear together.

In [ ]:
#YOUR CODE STARTS HERE#

rulers, knights = set(), set()

for character in all_characters:
    if (character.startswith('King') or character.startswith('Queen')) and character != "King" and character != "Queen":
        monarch_name = character.split()[1] if character.split()[1] != "of" else character
        rulers.add(monarch_name)
    elif character.startswith('Knight') or character.startswith('Sir'):
        name = character.split()[1] if character.split()[1] != "of" else character
        knights.add(name)

insieme_2 = {tuple([i, j]) for i in rulers for j in knights}

my_list = []

for tupla in tqdm(insieme_2):
    rating = len(df[df["Texts"].str.contains(tupla[0]) & df["Texts"].str.contains(tupla[1])].index)
    if rating != 0:
        my_list.append((tupla[0],  tupla[1], rating))

rating_df = pd.DataFrame(my_list)
rating_df.columns = ['rulers', 'knights', "ratings"]; print()

print("\n rating_df.shape :", rating_df.shape); print()
print(tabulate(rating_df.head(4), headers='keys', tablefmt = "heavy_outline", numalign='center', stralign="center"))
print()

#YOUR CODE ENDS HERE#
#THIS IS LINE 30#

100%|██████████| 1716/1716 [00:12<00:00, 132.83it/s]



 rating_df.shape : (685, 3)

     rulers    knights    ratings
--  --------  ---------  ---------
0    Pelles    Bliant        2
1     Bors    Aglovale       4
2     Bors      Ector       32
3   Pelleas    Dinadan       2



### 2.1.2
Inspect the dataset:

1. For each field, print the minimum and maximum values.

2. Print also the rows of the dataset where `Sir Accolon` appears.


In [ ]:
#YOUR CODE STARTS HERE#


s1 = rating_df.loc[rating_df['ratings'].idxmin()].to_frame().T
s2 = rating_df.loc[rating_df['ratings'].idxmax()].to_frame().T; print("")

print(tabulate(pd.concat([s1, s2], axis=0), headers='keys', tablefmt = "heavy_outline", numalign='center', stralign="center"))

print(); print()
print(tabulate(rating_df[rating_df["knights"] == "Accolon"], headers='keys', 
               tablefmt = "heavy_outline", numalign='center', stralign="center")); print()


#YOUR CODE ENDS HERE#
#THIS IS LINE 15#


      rulers    knights    ratings
---  --------  ---------  ---------
 7    Morgan     Mador        1
233   Arthur   Launcelot     204


      rulers     knights    ratings
---  ---------  ---------  ---------
62      Lot      Accolon       1
174   Uriens     Accolon       5
322  Guenever    Accolon       1
399   Arthur     Accolon      10
461   Morgan     Accolon       9
591  Pellinore   Accolon       1



<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 2.1.3
Load the dataset into the appropriate scikit-surprise structure.

In [ ]:
#YOUR CODE STARTS HERE#


reader = Reader(rating_scale=([0.5, 5]))
data =  Dataset.load_from_df(rating_df, reader=reader)



#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

### 2.1.4
Initialize a `scikit-surprise` `KFold` object with 3-folds.

In [ ]:
#YOUR CODE STARTS HERE#

kf = model_selection.KFold(n_splits = 3)





#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

### 2.1.5
Define **all** the algorithms you are going to use

In [ ]:
#YOUR CODE STARTS HERE#


all_algorithms = {"normal_predictor" : NormalPredictor, # no parameter configurations
          "baseline_only" :  BaselineOnly,
          "knnbasic" : KNNBasic,
          "knnwmeans" : KNNWithMeans,
          "knnwz" : KNNWithZScore,
          "knnbaseline" : KNNBaseline,
          "svd" : SVD,
          "svdpp" : SVDpp,
          "nmf" : NMF,
          "slope_One" : SlopeOne,     # # no parameter configurations
          "co_Clustering": CoClustering }




#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

### 2.1.6
Define the parameter configurations for each selected algorithm.

Each configuration must be a python `dict`.

Ensure that the definition meets the requirements of Part 2, but is also as minimal as possible (the fewer parameters you define, the better).

> Tip: dictionaries can be passed to methods using `**`. Example:

```
def method_name(param1, param2):
    return param1+param2
py_dict = {param1: 4, param2:2}
method_name(**py_dict) #gives 6
```



In [ ]:
#YOUR CODE STARTS HERE#

iterazioni = 25
baselines_estimation_method = "als"
similarity_measure = "cosine"
item_item_similarity = False

baseline = {'method': baselines_estimation_method,  'n_epochs': iterazioni}
knn_fam = {"sim_options" : {'name': similarity_measure, 'user_based': item_item_similarity, 'min_support': 3}}    ## KNN 
gen_options = {"n_epochs" : iterazioni}  # SVD, SVDPP, NMF, coClustering



list_of_dict = [baseline, knn_fam, knn_fam , knn_fam, knn_fam, gen_options, gen_options, gen_options, gen_options]




#YOUR CODE ENDS HERE#
#THIS IS LINE 30#

### 2.1.7
Print the number of CPU cores belonging to the machine on which Colab is running.

In [ ]:
#YOUR CODE STARTS HERE#


cores = multiprocessing.cpu_count()    # count the number of cores in a computer
print("number of CPU cores :", cores)



#YOUR CODE ENDS HERE#
#THIS IS LINE 10#

number of CPU cores : 2


Run a **3-Folds**-cross-validation for all the selected algorithms, using the parameters configuration that you selected for each algorithm.
Make sure that the cross-validation gives in output the metric measures for each split, as well as fit and test times.

[comment]: <> (#SHOW_CELL#)

In [ ]:
#YOUR CODE STARTS HERE#
c, my_list = 0, []

for k,v in all_algorithms.items():

  if k == "normal_predictor" or k == "slope_One":
    current_algo = v()
    x = model_selection.cross_validate(current_algo, data, measures=['mse'], cv=kf, n_jobs = -1, verbose=True)
    my_list.append((k,x)); print("\n**************************************************\n")
  elif k == "baseline_only": 
    current_algo = v(list_of_dict[c])
    x = model_selection.cross_validate(current_algo, data, measures=['mse'], cv=kf, n_jobs = -1, verbose=True)
    my_list.append((k,x)); c+=1; print("\n**************************************************\n")
  else:
    current_algo = v(**list_of_dict[c])
    x = model_selection.cross_validate(current_algo, data, measures=['mse'], cv=kf, n_jobs = -1, verbose=True)
    my_list.append((k,x)); c+=1; print("\n**************************************************\n")

#YOUR CODE ENDS HERE#
#THIS IS LINE 20#

Evaluating MSE of algorithm NormalPredictor on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MSE (testset)     249.5811316.993365.5858 210.7201106.2515
Fit time          0.00    0.00    0.00    0.00    0.00    
Test time         0.01    0.00    0.00    0.00    0.00    

**************************************************

Evaluating MSE of algorithm BaselineOnly on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MSE (testset)     271.9643154.9045157.2492194.706054.6383 
Fit time          0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    

**************************************************

Evaluating MSE of algorithm KNNBasic on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
MSE (testset)     274.8665224.572884.8098 194.749780.4050 
Fit time          0.00    0.00    0.00    0.00    0.00    
Test time         0.03    0.02    0.01    0.02    0.01    

***************

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 2.1.8
Rank all recommendation algorithms you tested according to the mean of the Mean Squared Error metric value: from the worst to the best algorithm.

Print out the ranking: algorithm name and MSE value.

In [ ]:
#YOUR CODE STARTS HERE#


lista = [(i[0], round(i[1]["test_mse"].mean(),5)) for i in my_list]
lista = sorted(lista, key=lambda x: x[1], reverse=True)

print()
print(tabulate(lista, headers = ["Algorithm","MSE_mean"]))
print()




#YOUR CODE ENDS HERE#
#THIS IS LINE 30#


Algorithm           MSE_mean
----------------  ----------
normal_predictor     211.738
co_Clustering        196.634
svdpp                196.275
knnbasic             195.093
baseline_only        194.767
knnwmeans            194.707
slope_One            194.639
knnwz                194.467
svd                  194.292
knnbaseline          194.035
nmf                  193.672



<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

### 2.1.9
Select the algorithm with the best result in the previous test.

You must test a maximum of **31** possible configurations for the selected recommendation algorithm. The number of parameters specified for the various configurations must be at least 2* and no more than 5*. Also, disregard configuration limitations described at the start of Part 2.

You must obtain the best configuration among all configurations, considering the Root Mean Squared Error metric calculated on a cross-validation of **5** folds.

1. Define the configuration dictionary that will be used for parameter optimisation.
2. Find a model configuration that offers the best possible performance within the given constraints. Print this configuration.

The resulting solution must exceed the default configuration according to the Mean Absolute Error metric.

> **If a parameter is itself composed of several parameters (e.g. if it is a dictionary), each will be counted separately when calculating the total number of attributes to be optimised.

In [ ]:
#YOUR CODE STARTS HERE#





#YOUR CODE ENDS HERE#
#THIS IS LINE 30#

<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>

## Part 2.2

### 2.2.1
Consider this scenario:

* There are $n$ users and $m$ items.
* The items are divided into two groups $I_A$ and $I_B$.
* Users can like (rating $1$) all items in group $I_A$ and dislike (rating $0$) those in group $I_B$, or vice versa, but no intermediate case; thus users can also be divided into users in group $U_A$ and users in group $U_B$.
* Suppose we have all $n$ x $m$ ratings.

Now, consider this:

* A new user $u$ is added and we record his preference of an item $i$ from group $I_A$ (rating $1$).

> What will be the estimated rating of an item $a \in I_A, a \neq i$ for user $u$ if we use user-based collaborative filtering? What will be the rating of item $b \in I_B$ instead?

> If the user adds that they do not like an item $j$ belonging to group $B$, how would the above ratings change ($b \neq j$)?

**Use at most 3 sentences.**

----------YOUR TEXT STARTS HERE----------


<div style="page-break-after: always; visibility: hidden">
\pagebreak
</div>


# Submission instructions

Follow the instructions precisely
1. Run the whole notebook up to this point
  * Click on this text cell and go to Runtime > Run before
  * It is essential that the notebook can be run from start to finish without us having to change anything.
2. Download the current notebook
  * File -> Download -> Download .ipynb
3. Upload the downloaded notebook in the current runtime
  * In the left sidebar menu, open the last tab (Files)
  * Click on the first button under the File title (Page with arrow pointing up)
  * In the new window, upload the notebook just downloaded



4. To turn the notebook into a pdf you need to install a Tex extension. Simply run the next cell of code:

In [ ]:
#REMOVE_CELL#
!apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
texlive-fonts-recommended is already the newest version (2019.20200218-1).
texlive-plain-generic is already the newest version (2019.202000218-1).
texlive-xetex is already the newest version (2019.20200218-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


5. Now, run the next cell to set some cell tags in the notebook file.

In [ ]:
#REMOVE_CELL#
import nbformat as nbf
from glob import glob
notebooks = glob("/content/**/*.ipynb", recursive=True)

text_search_dict = ["YOUR TEXT STARTS HERE","#YOUR CODE STARTS HERE#"]

for ipath in notebooks:
    print(ipath)
    ntbk = nbf.read(ipath, nbf.NO_CONVERT)

    for cell in ntbk.cells:
        cell_type = cell['cell_type'] == "code"
      
        cell_tags = cell.get('metadata', {}).get('tags', [])
        cell_tags = set(cell_tags)

        if cell_type == 0: #markdown
          if (cell['source'][0]!="#" and text_search_dict[cell_type] not in cell['source'] and "pagebreak" not in cell['source']) or "#REMOVE_CELL#" in cell['source']:
            cell_tags.add("remove_cell")

        else: #code
          #if text_search_dict[cell_type] not in cell['source'] or "#REMOVE_CELL#" in cell['source']:
          if "#REMOVE_CELL#" in cell['source']:
            cell_tags.add("remove_cell")
          elif "#REMOVE_OUTPUT#" in cell['source']:
            cell_tags.add("remove_output")
        
        if len(cell_tags) > 0:
            cell['metadata']['tags'] = list(cell_tags)

    nbf.write(ntbk, ipath)

6. Now, run the next cell to transform the notebook to PDF without the code cells.
  * The file will appear in the menu on the left (refresh if needed). The file should be named `DMT2023_HW2.pdf`
  * Rename this file **Surname1_Surname2_DMT2023HW2_report.pdf** (e.g. Becchetti_Siciliano_DMT2023HW2_report.pdf). Sort the surnames alphabetically.
  * Click on it and using the right button or the three dots at its side, click on download to transfer it to your local PC
  

In [ ]:
#REMOVE_CELL#
!jupyter nbconvert /content/DMT2023_HW2.ipynb --to pdf --TagRemovePreprocessor.remove_cell_tags='{"remove_cell"}' -TagRemovePreprocessor.remove_all_outputs_tags='{"remove_output"}'

[NbConvertApp] Converting notebook /content/DMT2023_HW2.ipynb to pdf
[NbConvertApp] Writing 121667 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 130198 bytes to /content/DMT2023_HW2.pdf


7. Now, rename the downloaded notebook **Surname1_Surname2_DMT2023_HW2_notebook.ipynb** (e.g. Becchetti_Siciliano_DMT2023_HW2_notebook.ipynb). Sort the surnames alphabetically.

8. Now that you have both files, you need to upload them to Classroom. Only the student with the surname that comes first in alphabetical order **must** upload the files. Only the files uploaded by this student will be graded.